# Thermal expansion of an undrained porous solid

*[Written by: Maxime Pierre, 2024]* \
\
Let us come back to our first example and complicate it a bit: we will consider an undrained, unconfined thermoporoelastic solid subject to a heat source. \
Density of water will be taken as a linear function of temperature and pressure (from IAWPS data): 
$$
\rho_f = 1000 + 0.487p - 0.0284(T-T_0).
$$
The usual thermoporoelastic equations read:
$$
\begin{array}{}
\sigma = K(\varepsilon)\varepsilon - bp - \alpha_s KT, \\
\phi-\phi_0 = b\varepsilon - \frac{p}{N} - 3\alpha_s(b-\phi)T, \\
S - S_0 = \alpha_s K \varepsilon - 3\alpha_s(b-\phi)p + C_s\frac{T}{T_0}.
\end{array}
$$



In [16]:
import numpy as np
import matplotlib.pyplot as plt

# Material constants
K = 1e9
b = 0.6
K_f = 2e9
N = 1e11
phi_0 = 0.3
alpha_s = 1e-5
alpha_w = 2.1e-4

# Loading
sigma_ = -1e6

def rho_f(p,T):
    1000 + 0.487*p - 0.0284*T

def F(X):
    eps, p, T = X
    return np.array([K * eps - b*p + alpha_s*T, 
                     (1+p/K_f-2.84e-5*dT)*(phi_0+b*eps-p/N) -phi_0 + alpha_s*(b-phi_0)*dT])

def J(X):
    eps, p = X

    dsig_deps = 3*K_0*eps**2
    dsig_dp = -b
    dmf_deps = b * (1+p/K_f)
    dmf_dp = -1/N * (1+p/K_f) + 1/K_f * (phi_0+b*eps-p/N)
    return np.array([ [dsig_deps, dsig_dp], [dmf_deps, dmf_dp] ])
    
def newton_method(residu, jacobian, initial_guess, tolerance, max_iteration=50):
    X_n = initial_guess
    X = [initial_guess]
    iteration = 0
    
    while np.linalg.norm(residu(X_n)) > tolerance and iteration < max_iteration:
        
        X_n -= np.dot(np.linalg.inv(jacobian(X_n)), residu(X_n))
        
        X.append(X_n.copy())
        iteration += 1
        print("Iteration {}: X_{} = {}, residual norm = {}".format(iteration, iteration, X_n, np.linalg.norm(residu(X_n))))
        
    if np.linalg.norm(residu(X_n)) > tolerance: # Not converged
        print("Did not converge after {} iterations.".format(iteration))
    else: # Converged
        print("Converged in {} iterations.".format(iteration))
        
    return np.array(X)

X_0 = [0,0]
eps = 1e-12

result = newton_method(lambda X : F(X, [sigma_, 0]), J, X_0, eps)

Iteration 1: X_1 = [-4.13888889e-04  1.66666667e+06], residual norm = 0.7090082733193719
Iteration 2: X_2 = [-4.13541664e-04  1.66666549e+06], residual norm = 1.496635377407074e-06
Iteration 3: X_3 = [-4.13541664e-04  1.66666549e+06], residual norm = 1.1641532182693481e-10
Iteration 4: X_4 = [-4.13541664e-04  1.66666549e+06], residual norm = 0.0
Converged in 4 iterations.
